In [84]:
import os
from osgeo import gdal, osr
import numpy as np
from osgeo.gdalconst import GDT_Float32

In [123]:
input_file = '../bioclim2000-10/wc2.1_2.5m_2000_1.grd'
output_file = '../bioclim2000-10/wc2.1_2.5m_2000_1.tiff'
output_file_asc = '../bioclim2000-10/wc2.1_2.5m_2000_1.asc'

from osgeo import gdal, osr
def translate_to_tiff(input_file, output_file):
    """
    Function to convert data from asc, grd format to GeoTiff
    """
    drv = gdal.GetDriverByName('GTiff')
    ds_in = gdal.Open(input_file)
    ds_out = drv.CreateCopy(output_file, ds_in)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    ds_out.SetProjection(srs.ExportToWkt())
    matrix = ds_in.ReadAsArray()
    ds_in = None
    ds_out = None
    return matrix

def bioclim_values_to_tiff(input_file, output_file, values):
    in_data, out_data = None, None
    in_data = gdal.Open(input_file)
    if in_data is None:
        print ('Unable to open %s' % raster_input)
        return None

    # read in data from first band of input raster
    band1 = in_data.GetRasterBand(1)
    rows = in_data.RasterYSize
    cols = in_data.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)

    driver = in_data.GetDriver()
    number_of_bands = values.shape[0]
    out_data = driver.Create(output_file, cols, rows, number_of_bands, GDT_Float32)
    for i in range(number_of_bands):
        out_band = out_data.GetRasterBand(i+1)
        out_band.WriteArray(values[i, :, :])
    out_band.FlushCache()
    out_band.SetNoDataValue(-32767.)

    out_data.SetGeoTransform(in_data.GetGeoTransform())
    out_data.SetProjection(in_data.GetProjection())
    del out_data
    return output_file
def bioclim_values_to_separate_tiff(input_file, output_file, values):
    in_data, out_data = None, None
    in_data = gdal.Open(input_file)
    if in_data is None:
        print ('Unable to open %s' % raster_input)
        return None

    # read in data from first band of input raster
    band1 = in_data.GetRasterBand(1)
    rows = in_data.RasterYSize
    cols = in_data.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)

    driver = in_data.GetDriver()
    out_data = driver.Create(output_file, cols, rows, 1, GDT_Float32)

    out_band = out_data.GetRasterBand(1)
    out_band.WriteArray(values)
    out_band.FlushCache()
    out_band.SetNoDataValue(-32767.)

    out_data.SetGeoTransform(in_data.GetGeoTransform())
    out_data.SetProjection(in_data.GetProjection())
    del out_data
    return output_file



In [124]:
grd_files = [file for file in os.listdir('../bioclim2000-10/') if '.grd' in file]
list_of_matrix = []
for file in grd_files:
    input_file_grd = os.path.join('../bioclim2000-10', file)
    output_file_tiff = os.path.join('../bioclim2000-10',\
                               os.path.splitext(file)[0]+'.tiff')
    m = translate_to_tiff(input_file_grd, output_file_tiff)
    list_of_matrix.append(m)
    
    

# compute mean without nan values along 0 axis
mean_bioclim = np.nanmean(list_of_matrix, axis=0)

/Users/mikhailgasanov/opt/anaconda3/envs/maxvol_soil_sampling/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]


In [125]:
output_bioclim_mean = os.path.join('../bioclim2000-10/', 'bioclim_mean.tiff')
# save mean_bioclim as GeoTiff - all bands
out_file = bioclim_values_to_tiff(input_file=output_file_tiff, \
                                  output_file=output_bioclim_mean, \
                                  values = mean_bioclim)


In [126]:
# Save biolimatic variables as separated GeoTiff files
for index, bioclim in enumerate(mean_bioclim):
    data_to_tiff = mean_bioclim[index, :,:]
    output_bioclim_mean = os.path.join('../bioclim2000-10/', f'bioclim_mean_{index}.tiff')
    out_file = bioclim_values_to_separate_tiff(input_file=output_file_tiff, \
                                  output_file=output_bioclim_mean, \
                                  values = data_to_tiff)

In [127]:
# translate GeoTiff to ASCII
for index in range(19):
    asc_file = os.path.join('../bioclim2000-10', f'bioclim_mean_{index}.asc')
    tiff_file = os.path.join('../bioclim2000-10', f'bioclim_mean_{index}.tiff')
    ds = gdal.Open(tiff_file)
    ds = gdal.Translate(asc_file, ds)
    ds = None

In [128]:
# asc files
asc_files=[]
for file in os.listdir('../bioclim2000-10/'):
    if 'asc' in file:
        print(file)
        asc_files.append(file)

bioclim_mean_10.asc
bioclim_mean_9.asc.aux.xml
bioclim_mean_11.asc
bioclim_mean_13.asc
bioclim_mean_17.asc.aux.xml
bioclim_mean_8.asc
bioclim_mean_9.asc
bioclim_mean_0.asc.aux.xml
bioclim_mean_12.asc
bioclim_mean_16.asc
bioclim_mean_12.asc.aux.xml
bioclim_mean_5.asc.aux.xml
bioclim_mean_17.asc
bioclim_mean_15.asc
bioclim_mean_14.asc
bioclim_mean_18.asc.aux.xml
bioclim_mean_6.asc.aux.xml
bioclim_mean_11.asc.aux.xml
bioclim_mean_3.asc.aux.xml
bioclim_mean_14.asc.aux.xml
bioclim_mean_4.asc.aux.xml
bioclim_mean_13.asc.aux.xml
bioclim_mean_1.asc.aux.xml
bioclim_mean_16.asc.aux.xml
bioclim_mean_8.asc.aux.xml
bioclim_mean_2.asc
bioclim_mean_3.asc
bioclim_mean_18.asc
bioclim_mean_15.asc.aux.xml
bioclim_mean_1.asc
bioclim_mean_0.asc
bioclim_mean_2.asc.aux.xml
bioclim_mean_10.asc.aux.xml
bioclim_mean_4.asc
bioclim_mean_5.asc
bioclim_mean_7.asc.aux.xml
bioclim_mean_7.asc
bioclim_mean_6.asc


In [129]:
os.makedirs(os.path.join('..', 'bioclim2000-10', 'asc_files'))

In [130]:
import shutil

In [133]:
# copy all file into separate folder
!cp ../bioclim2000-10/*.asc* ../bioclim2000-10/asc_files/

In [135]:
shutil.make_archive('../bioclim2000-10/asc_bioclim_files', 'zip', '../bioclim2000-10/asc_files/')

'/Users/mikhailgasanov/Documents/GIT/herscleum_sosnowskyi/bioclim2000-10/asc_bioclim_files.zip'